This script is used to test the modules.

In [9]:
%matplotlib inline

import numpy as np
import scipy
from netCDF4 import Dataset
import matplotlib
import matplotlib.pyplot as plt
import os
import statistics

In [11]:
if __name__=="__main__":
    # sample
    ################################   Input zone  ######################################
    # specify data path
    datapath_ERAI = '/home/yang/workbench/Core_Database_AMET_OMET_reanalysis/ERAI/postprocessing'
    datapath_ORAS4 = '/home/yang/workbench/Core_Database_AMET_OMET_reanalysis/ORAS4/postprocessing'
    datapath_ERAI_fields = '/home/yang/workbench/Core_Database_AMET_OMET_reanalysis/ERAI/regression'
    #####################################################################################
    print ('*********************** extract variables *************************')
    dataset_ERAI_fields_SIC_SST_SLP = Dataset(os.path.join(datapath_ERAI_fields,'surface_ERAI_monthly_regress_1979_2016.nc'))
    dataset_ERAI_AMET = Dataset(os.path.join(datapath_ERAI,'model_daily_075_1979_2016_E_zonal_int.nc'))
    dataset_ORAS4_OMET = Dataset(os.path.join(datapath_ORAS4,'oras4_model_monthly_orca1_E_zonal_int.nc'))
    # extract time series from 1979 to 2014
    # from 20N - 90N
    AMET_ERAI_reverse = dataset_ERAI_AMET.variables['E'][:-2,:,:]/1000 # from Tera Watt to Peta Watt
    OMET_ORAS4 = dataset_ORAS4_OMET.variables['E'][21:,:,180:]/1000 # from Tera Watt to Peta Watt # start from 1979
    year_ORAS4 = dataset_ORAS4_OMET.variables['year'][21:]    # from 1979 to 2014
    month_ORAS4 = dataset_ORAS4_OMET.variables['month'][:]
    latitude_OMET_ORAS4 = dataset_ORAS4_OMET.variables['latitude_aux'][180:]
    latitude_AMET_ERAI_reverse = dataset_ERAI_AMET.variables['latitude'][:]
    # since OMET is from 20N - 90N, AMET is from 90N to 20N, we have to reverse it
    # for interpolation, x should be monotonically increasing
    latitude_AMET_ERAI = latitude_AMET_ERAI_reverse[::-1]
    AMET_ERAI = AMET_ERAI_reverse[:,:,::-1]
    print (AMET_ERAI.shape)
    print (type(OMET_ORAS4))

*********************** extract variables *************************
(36, 12, 95)
<class 'numpy.ma.core.MaskedArray'>


In [14]:
    print ('*******************  interpolation for regression   **********************')
    # interpolate OMET on the latitude of AMET
    OMET_ORAS4_interp = np.zeros((len(year_ORAS4),len(month_ORAS4),len(latitude_AMET_ERAI)),dtype=float)
    for i in np.arange(len(year_ORAS4)):
        for j in np.arange(len(month_ORAS4)):
            ius = scipy.interpolate.interp1d(latitude_OMET_ORAS4, OMET_ORAS4[i,j,:], kind='slinear',
                                             bounds_error=False, fill_value=0.0)
            OMET_ORAS4_interp[i,j,:] = ius(latitude_AMET_ERAI.data)
    #print (OMET_ORAS4_interp)

*******************  interpolation for regression   **********************


In [15]:
    print ('*******************  postprocess with statistical tool  *********************')
    stat_AMET_ERAI = statistics.operator(AMET_ERAI)
    stat_AMET_ERAI.anomaly()
    stat_AMET_ERAI.lowpass()
    stat_OMET_ORAS4 = statistics.operator(OMET_ORAS4_interp)
    stat_OMET_ORAS4.anomaly()
    stat_OMET_ORAS4.detrend()
    stat_OMET_ORAS4.lowpass(obj='detrend')

*******************  postprocess with statistical tool  *********************
The input data has the dimension of month.
The output anomaly time series only contains one dimension for time!
The input data has the dimension of month.
The output anomaly time series only contains one dimension for time!


array([[-0.02349856, -0.03247318, -0.03698061, ..., -0.00111906,
        -0.00055318,  0.        ],
       [-0.02544719, -0.03514104, -0.04068311, ..., -0.00083052,
        -0.00035005,  0.        ],
       [-0.02350534, -0.03398948, -0.04027839, ..., -0.00053838,
        -0.00014856,  0.        ],
       ...,
       [-0.00222099, -0.00591381, -0.01023698, ...,  0.00104718,
         0.00070661,  0.        ],
       [-0.00962374, -0.01015496, -0.01159175, ...,  0.00109981,
         0.00079336,  0.        ],
       [ 0.00442281,  0.00496903,  0.00514259, ...,  0.00105569,
         0.00075247,  0.        ]])